# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
## Bin packing problem: 
## Даны: 
* $[w_1, \dots , w_k], w_i \in [0, 1]$ — веса грузов  
* $n_{bins}$ — количество корзин грузоподъемностью 1.

Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing) и затем содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [33]:
def able_to_pack(weights: list, bin_capacities) -> bool:
        return weights == [] or any( 
            able_to_pack(weights[:-1], bin_capacities[:i] + [capacity - weights[-1]] + bin_capacities[(i + 1):] ) 
            for i, capacity in enumerate(bin_capacities) if capacity >= weights[-1] 
        )


def solve_bp_decision(weights: list, n_bins: int) -> bool:
    
    if any(weight > 1 for weight in weights):
        return False
    
    return able_to_pack(sorted(weights), [1.0] * n_bins)


def solve_bp_evaluation(weights: list) -> int:
    assert all(0 <= weight <= 1 for weight in weights)
    lower_bound = 0  # недостижимо
    upper_bound = len(weights)  # достижимо
    
    while upper_bound - lower_bound > 1:
        mid = (upper_bound + lower_bound) // 2
        if solve_bp_decision(weights, mid):
            upper_bound = mid
        else:
            lower_bound = mid
    return upper_bound


# returns number of corresponding bin
# for each weight in input in same order
# e.g. [0.8, 0.09, 0.4, 0.7] -> [1,  2,  2, 3]
def solve_bp_search(weights: list) -> list:
    optimal_ans = solve_bp_evaluation(weights)
    n = len(weights)
    
    ans = []
    capacities = [1.0] * optimal_ans
    for cur_w in range(n):
        for bin_i in range(n):
            if capacities[bin_i] >= weights[cur_w]:
                capacities[bin_i] -= weights[cur_w]
                if able_to_pack(weights[cur_w + 1:], capacities):
                    ans.append(bin_i + 1)
                    break
                else:
                    capacities[bin_i] += weights[cur_w]

    return ans

In [32]:
import unittest


class TestBinPackingSolutions(unittest.TestCase):

    def test_decision_1(self):
        weights = [0.8, 0.09, 0.4, 0.7]
        self.assertTrue(solve_bp_decision(weights, 3))

    def test_decision_2(self):
        weights = [0.8, 0.09, 0.4, 0.7]
        self.assertFalse(solve_bp_decision(weights, 2))

    def test_evaluation_1(self):
        weights = [0.8, 0.09, 0.4, 0.7]
        self.assertEqual(solve_bp_evaluation(weights), 3)
    
    def test_evalutation_2(self):
        weights = [0.3000000001, 0.95, 0.06, 0.7, 0.8]
        self.assertEqual(solve_bp_evaluation(weights), 4)

    def test_search(self):
        test_cases = [[0.8, 0.09, 0.4, 0.7],
                      [1, 1, 1, 1, 0, 0, 0],
                      [0.01, 0.8, 0.19]]
        for subtest_i, weights in enumerate(test_cases):
            with self.subTest(i=subtest_i):
                optimal_bin_n = solve_bp_evaluation(weights)
                solution = solve_bp_search(weights)
                
                self.assertTrue(all(1 <= bin_num <= optimal_bin_n for bin_num in solution), 
                                "too many bins")
                
                bin_weights = [0] * optimal_bin_n
                for i in range(optimal_bin_n):
                    bin_weights[solution[i] - 1] += weights[i]
                
                self.assertTrue(all(0 <= total_weight <= 1 for total_weight in bin_weights),
                                "incorrect packing")

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.

.

.

.

.


----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK
